# Swarm-Induced Graph

Using 'robot' positions at each timestep, compute pair-wise distances.
Robots within a threshold distance 

Galen Wilkerson

(using liquidfun robotics swarm by Calum Emrie, University of Edinburgh)

In [214]:
import pandas as pd

import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

#If you want all graphs to be drawn inline, then you can set a global parameter.
#from nxpd import draw
#from nxpd import nxpdParams
#nxpdParams['show'] = 'ipynb'

from IPython import display
from ipywidgets import interact

import scipy.spatial.distance
import networkx as nx

In [215]:
# read position data from file
df = pd.read_csv('./example_freeze', sep=' ', header=None)

# last column is junk
y = df.pop(500)

In [216]:
row = df.loc[0].values
row = row.reshape(250,2)

#plt.scatter(row[:,0], row[:,1], marker='.');


def update(timestamp = widgets.IntSlider(min=0,max=len(df),step=1,value=0), 
           distance_threshold = widgets.FloatSlider(min=0.01,max=7.5,step=.01,value=1)):
    
    row = df.loc[timestamp].values
    row = row.reshape(250,2)


    plt.figure(figsize=[10,10])
    plt.clf()

    plt.subplot(221)
    plt.scatter(row[:,0], row[:,1], marker='.');
    plt.title('positions')

    plt.subplot(222);
    
    # calculate pairwise distances


    X = row
    Y = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(X, 'euclidean'))
    adj = Y < distance_threshold
    
    adj_mat = Y * adj.T
    
    G=nx.from_numpy_matrix(adj_mat)

    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())

    pos = dict(zip(range(len(row)), row))
    
    
    nx.draw(G, pos=pos, 
            node_size=3, 
            node_color = 'k',
            #edge_color='grey', 
            alpha=.5,
            edgelist=edges, 
            edge_color=1 - np.array(weights)/max(weights),
            edge_cmap=plt.cm.Blues);
    
    plt.title('connections')
    
    plt.subplot(223)
    plt.title('degree histogram')
    plt.scatter(range(len(nx.degree_histogram(G))),nx.degree_histogram(G), marker='.');
    
    plt.subplot(224)
    plt.title('distance histogram')
    plt.hist(Y.flatten(), bins = 50);
    
interact(update);


interactive(children=(IntSlider(value=0, description='timestamp', max=102), FloatSlider(value=1.0, description…